In [3]:
#!/usr/bin/env python
# coding: utf-8

import torch
import os
import time
import pandas as pd
import h3
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, DataCollatorWithPadding
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Paths and settings
state_to_do = 'PUNJAB'
address_data_path = f'/home/uddeshya.singh/DATA/PUNJAB_train/PUNJAB_with_grid_ids_train.csv'
fine_tuned_lm_trained_save_dir = '/home/uddeshya.singh/Model_test/Fine-tuned-save'
fine_tuned_lm_trained_out_dir = '/home/uddeshya.singh/Model_test/Fine-tuned-out'
model_dir = '/home/uddeshya.singh/Models/pan_india_train_threshold_save_dir/'

# Load tokenizer
pretrained_tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load and preprocess data
print('Starting now ', time.strftime('%X %x %Z'))
df_train = pd.read_csv(address_data_path)

print('Completed loading ', time.strftime('%X %x %Z'))

# Compute grid IDs
def compute_grid_ids(row, res):
    return h3.geo_to_h3(row['lookup_lat'], row['lookup_lng'], res)

df_train['grid_id_res9'] = df_train.apply(compute_grid_ids, axis=1, res=9)
df_train['grid_id_res10'] = df_train.apply(compute_grid_ids, axis=1, res=10)

# Convert to Hugging Face Dataset
address_data_train = Dataset.from_pandas(df_train)
print(address_data_train)

# Tokenize function
def tokenize(batch):
    return pretrained_tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)

# Clean dataset
def clean_dataset(batch):
    if pd.isnull(batch['cleaned_address']) or batch['cleaned_address'] == '':
        return False
    return True

print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.filter(clean_dataset)
print('completed now ', time.strftime('%X %x %Z'))

# Encode columns
address_data_train = address_data_train.remove_columns(['addr_hash', 'lookup_lat', 'lookup_lng' , 'grid_id'])
address_data_train = address_data_train.class_encode_column("grid_id_res10")
address_data_train = address_data_train.class_encode_column("grid_id_res9")

# Tokenize data
train_dataset = address_data_train.map(tokenize, batched=True, batch_size=100000, num_proc=7)

# Set format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "grid_id_res10", "grid_id_res9"])

# Collate function
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels_res10 = torch.tensor([item['grid_id_res10'] for item in batch])
    labels_res9 = torch.tensor([item['grid_id_res9'] for item in batch])
    
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=pretrained_tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    
    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'labels_res10': labels_res10,
        'labels_res9': labels_res9
    }



cuda
Starting now  15:48:38 07/29/24 IST
Completed loading  15:48:50 07/29/24 IST
Dataset({
    features: ['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id', 'grid_id_res9', 'grid_id_res10'],
    num_rows: 6913138
})
starting now  15:51:44 07/29/24 IST


  0%|          | 0/6914 [00:00<?, ?ba/s]

completed now  15:52:54 07/29/24 IST


Flattening the indices:   0%|          | 0/6914 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/6914 [00:00<?, ?ba/s]

...

#6:   0%|          | 0/10 [00:00<?, ?ba/s]

Some weights of the model checkpoint at /home/uddeshya.singh/Models/pan_india_train_threshold_save_dir/ were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/Models/pan_india_train_threshold_save_dir/ and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a

starting now  15:59:23 07/29/24 IST


  0%|          | 239/3456559 [01:20<324:03:49,  2.96it/s]


KeyboardInterrupt: 

In [5]:
# Data loader with pin_memory and num_workers
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)

# Number of labels
num_labels_res10 = address_data_train.features["grid_id_res10"].num_classes
num_labels_res9 = address_data_train.features["grid_id_res9"].num_classes

# Multi-task model
class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_labels_res10, num_labels_res9):
        super(MultiTaskModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier_res10 = nn.Linear(self.hidden_size, num_labels_res10)
        self.classifier_res9 = nn.Linear(self.hidden_size, num_labels_res9)
    
    def forward(self, input_ids, attention_mask=None, labels_res10=None, labels_res9=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        logits_res10 = self.classifier_res10(pooled_output)
        logits_res9 = self.classifier_res9(pooled_output)

        loss = None
        if labels_res10 is not None and labels_res9 is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss_res10 = loss_fct(logits_res10.view(-1, num_labels_res10), labels_res10.view(-1))
            loss_res9 = loss_fct(logits_res9.view(-1, num_labels_res9), labels_res9.view(-1))
            loss = 0.5 * loss_res10 + 0.5 * loss_res9

        return loss, logits_res10, logits_res9

# Initialize model and optimizer
model = MultiTaskModel(model_dir, num_labels_res10, num_labels_res9)
model = nn.DataParallel(model)  # Use DataParallel to distribute across multiple GPUs
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 5

# Train model with DataParallel
def train_model(model, train_loader, optimizer, num_epochs, save_dir, out_dir):
    loss_values = []
    save_interval = 1000
    iteration = 0

    # Create directories if they do not exist
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader):
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            labels_res10 = batch['labels_res10'].to(device, non_blocking=True)
            labels_res9 = batch['labels_res9'].to(device, non_blocking=True)

            optimizer.zero_grad()
            loss, logits_res10, logits_res9 = model(input_ids, attention_mask, labels_res10, labels_res9)
            
            # Ensure loss is a scalar
            loss = loss.mean()
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            iteration += 1
            
            if iteration % save_interval == 0:
                model_save_path = f"{save_dir}/model_MM_checkpoint_{iteration}.pt"
                optimizer_save_path = f"{out_dir}/optimizer_MM_checkpoint_{iteration}.pt"
                torch.save(model.state_dict(), model_save_path)
                torch.save(optimizer.state_dict(), optimizer_save_path)
        
        avg_loss = total_loss / len(train_loader)
        loss_values.append(avg_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

    # Save the final model
    final_model_save_path = f"{save_dir}/MM_model.pt"
    torch.save(model.state_dict(), final_model_save_path)

    # Save the final optimizer state
    optimizer_save_path = f"{out_dir}/optimizer_MM.pt"
    torch.save(optimizer.state_dict(), optimizer_save_path)

    # Plot training loss
    plt.figure()
    plt.plot(range(1, num_epochs + 1), loss_values, marker='o', label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.savefig(f"{save_dir}/training_loss.png")
    plt.show()

    return loss_values

print('starting now ', time.strftime('%X %x %Z'))
start_time = time.time()
train_model(model, train_loader, optimizer, num_epochs, fine_tuned_lm_trained_save_dir, fine_tuned_lm_trained_out_dir)
end_time = time.time()
print('completed now ', time.strftime('%X %x %Z'))


Some weights of the model checkpoint at /home/uddeshya.singh/Models/pan_india_train_threshold_save_dir/ were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/Models/pan_india_train_threshold_save_dir/ and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a

starting now  16:02:34 07/29/24 IST


  0%|          | 227/864140 [01:52<117:34:45,  2.04it/s]Exception in thread Thread-3367:
Traceback (most recent call last):
  File "/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 289, in rebuild_storage_fd
    fd = df.detach()
  File "/home/uddeshya.singh/miniconda3/envs/uddeshy

KeyboardInterrupt: 

In [2]:
print('hi')

hi
